In [15]:
using MeshCatMechanisms
using MeshCat
using LinearAlgebra
using RigidBodyDynamics
using Plots
using GeometryTypes
using CoordinateTransformations
using TrajectoryOptimization

Load Kuka iiwa model (from URDF)

In [16]:
kuka = parse_urdf(Dynamics.urdf_kuka,remove_fixed_tree_joints=false)

model = Dynamics.kuka
model_d = rk3(model)
n,m = model.n, model.m

(14, 7)

Numerical type

In [17]:
T = Float64

Float64

In [18]:
function hold_trajectory(n,m,N, mech::Mechanism, q)
    state = MechanismState(mech)
    nn = num_positions(state)
    set_configuration!(state, q[1:nn])
    vd = zero(state.q)
    u0 = dynamics_bias(state)

    
    if length(q) > m
        throw(ArgumentError("system must be fully actuated to hold an arbitrary position ($(length(q)) should be > $m)"))
    end
    U0 = zeros(m,N)
    for k = 1:N
        U0[:,k] = u0
    end
    return U0
end


hold_trajectory (generic function with 1 method)

Initial and final states

In [19]:
x0 = zeros(n)

xf = zeros(n)
xf[1] = pi/2
xf[2] = pi/2;

Cost

In [20]:
Q = Diagonal([ones(7); ones(7)*100])
Qf = 1000.0*Diagonal(I,n)
R = 1e-2*Diagonal(I,m);

Solver options

In [21]:
verbose=false

opts_ilqr = iLQRSolverOptions{T}(verbose=true,
    iterations=300,
    live_plotting=:off)

opts_al = AugmentedLagrangianSolverOptions{T}(verbose=true,
    opts_uncon=opts_ilqr,
    iterations=20,
    cost_tolerance=1.0e-6,
    cost_tolerance_intermediate=1.0e-5,
    constraint_tolerance=1.0e-3,
    penalty_scaling=50.,
    penalty_initial=0.01)

opts_altro = ALTROSolverOptions{T}(verbose=true,
    resolve_feasible_problem=false,
    opts_al=opts_al,
    R_inf=0.01);

Terminal constraint

In [22]:
goal = goal_constraint(xf);

Problem

In [26]:
N = 51 # number of knot points
tf = 5.0
dt = tf/(N-1)# total time

U_hold = hold_trajectory(n,m,N, kuka, x0[1:7])
obj = LQRObjective(Q,R,Qf,xf,N) # objective with same stagewise costs
con = Constraints(N)
con[N] += goal

prob = Problem(model_d,obj,x0=x0, xf=xf, dt=dt, constraints=con)
initial_controls!(prob, U_hold); # initialize problem with controls

In [27]:
solve!(prob, opts_al);

iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
1     0       1.57079633  2479.763 
    iter cost          expected    z         α         ρ         dJ        grad      zero_count info                                              
____----------------------------------------------------------------------------------------------------------------------------------------------
    1     1494.3188748  1026.44524  0.960055  0.25      0e+00     985.4439  3.76957  0          
    2     922.58617409  596.549237  0.9584    0.25      0e+00     571.7327  3.377408 0          
    3     382.20076329  597.337196  0.904657  0.5       0e+00     540.3854  3.333207 0          
    4     351.16619232  197.343399  0.157262  0.5       0e+00     31.03457  2.922638 0          
    5     134.48247226  234.996656  0.922072  1         0e+00     216.6837  1.79147  0          
    6     121

Visualizer

In [31]:
kuka_visuals = URDFVisuals(Dynamics.urdf_kuka)
state = MechanismState(kuka)
world = root_frame(kuka)

# Create Visualizer
vis = Visualizer()
mvis = MechanismVisualizer(kuka, kuka_visuals, vis[:base])
# open(vis)
IJuliaCell(vis);

Create animation

In [32]:
q = [prob.X[k][1:7] for k = 1:N]
t = range(0,stop=tf,length=N)

setanimation!(mvis,t,q)